In [1]:
import tensorflow as tf
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
import csv
import numpy as np

In [2]:

"""
Global values
"""
folder = 'boxoffice_10'

len_dtcgum = pickle.load(open('../data/'+folder+'/len_dtcgum.pkl', 'rb'))

FEATURE_LEN = 5

MOVIE_NUM = len_dtcgum[5]
DIRECTOR_NUM = len_dtcgum[0]
GENRE_NUM = len_dtcgum[3]
TOPIC_NUM = 50
CAST_NUM = len_dtcgum[2]
USER_NUM = len_dtcgum[4]

print('movie:', MOVIE_NUM)
print('user:', USER_NUM)
print('director:', DIRECTOR_NUM)
print('genre:', GENRE_NUM)
print('topic:', TOPIC_NUM)
print('cast:', CAST_NUM)

"""
MOVIE_NUM = 1682
DIRECTOR_NUM = 1139
GENRE_NUM = 24
TOPIC_NUM = 50
CAST_NUM = 2894
USER_NUM = 943
"""

movie: 206
user: 3550
director: 194
genre: 20
topic: 50
cast: 474


'\nMOVIE_NUM = 1682\nDIRECTOR_NUM = 1139\nGENRE_NUM = 24\nTOPIC_NUM = 50\nCAST_NUM = 2894\nUSER_NUM = 943\n'

In [3]:
movie_rating = pickle.load(open('../data/'+folder+'/movie_rating.pkl', 'rb'))
movie_rating_test = pickle.load(open('../data/'+folder+'/movie_rating_test.pkl', 'rb'))
movie_rating_validation = pickle.load(open('../data/'+folder+'/movie_rating_validation.pkl', 'rb'))


In [4]:
# load data
def load_data(rating_file):
    user_list = []
    movie_list = []
    rating_list = []
    for movie_id in rating_file:
        for tup in rating_file[movie_id]:
            user_id = tup[0]
            rating = tup[1]
            user_list.append(user_id)
            movie_list.append(movie_id)
            rating_list.append(rating)
    
    return user_list, movie_list, rating_list

In [5]:
#load training data
total_train_user, total_train_movie, total_train_rating = load_data(movie_rating)
total_valid_user, total_valid_movie, total_valid_rating = load_data(movie_rating_validation)
total_test_user, total_test_movie, total_test_rating = load_data(movie_rating_test)
      

In [6]:
# load validation and test data
total_train_user_v = []
total_train_movie_v = []
total_train_rating_v = []
for movie_id in movie_rating:
    for tup in movie_rating[movie_id]:
        user_id = tup[0]
        rating = tup[1]
        total_train_user.append(user_id)
        total_train_movie.append(movie_id)
        total_train_rating.append(rating)


In [7]:
def load_data(user_name, movie_name, rating_name, last_same):
    user = pickle.load(open('../data/'+folder+'/'+user_name+last_same+'.pkl', 'rb'))
    movie = pickle.load(open('../data/'+folder+'/'+movie_name+last_same+'.pkl', 'rb'))
    rating = pickle.load(open('../data/'+folder+'/'+rating_name+last_same+'.pkl', 'rb'))
    return shuffle(user, movie, rating)

#load training data
d_user, d_movie, d_rating = load_data("train_user", "train_movie", "train_rating", '_d')
t_user, t_movie, t_rating = load_data("train_user", "train_movie", "train_rating", '_t')
g_user, g_movie, g_rating = load_data("train_user", "train_movie", "train_rating", '_g')   
c_user, c_movie, c_rating = load_data("train_user", "train_movie", "train_rating", '_c')

#load validation data
d_user_v, d_movie_v, d_rating_v = load_data("train_user", "train_movie", "train_rating", '_d_v')
t_user_v, t_movie_v, t_rating_v = load_data("train_user", "train_movie", "train_rating", '_t_v')
g_user_v, g_movie_v, g_rating_v = load_data("train_user", "train_movie", "train_rating", '_g_v')        
c_user_v, c_movie_v, c_rating_v = load_data("train_user", "train_movie", "train_rating", '_c_v')


In [8]:
U = tf.Variable(initial_value=tf.truncated_normal([USER_NUM, FEATURE_LEN]), name='users')
D = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, DIRECTOR_NUM]), name='directors')
T = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, TOPIC_NUM]), name='topics')
G = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, GENRE_NUM]), name='genures')
C = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, CAST_NUM]), name='casts')

"""
minvalue = 0
maxvalue = 10
U = tf.clip_by_value(U, minvalue, maxvalue)
D = tf.clip_by_value(D, minvalue, maxvalue)
T = tf.clip_by_value(T, minvalue, maxvalue)
G = tf.clip_by_value(G, minvalue, maxvalue)
C = tf.clip_by_value(C, minvalue, maxvalue)
"""

'\nminvalue = 0\nmaxvalue = 10\nU = tf.clip_by_value(U, minvalue, maxvalue)\nD = tf.clip_by_value(D, minvalue, maxvalue)\nT = tf.clip_by_value(T, minvalue, maxvalue)\nG = tf.clip_by_value(G, minvalue, maxvalue)\nC = tf.clip_by_value(C, minvalue, maxvalue)\n'

# generate the in the film matrix

### if the information is in the movie, the value will be 1, otherwise will be 0

In [9]:
d_result = tf.matmul(U, D)
t_result = tf.matmul(U, T)
g_result = tf.matmul(U, G)
c_result = tf.matmul(U, C)

In [10]:
d_result

<tf.Tensor 'MatMul:0' shape=(3550, 194) dtype=float32>

In [11]:
d_var = tf.Variable(initial_value=tf.random_normal([USER_NUM, DIRECTOR_NUM], mean=0,stddev=0.05), name='d_var')
t_var = tf.Variable(initial_value=tf.random_normal([USER_NUM, TOPIC_NUM], mean=0, stddev=0.05), name='t_var')
g_var = tf.Variable(initial_value=tf.random_normal([USER_NUM, GENRE_NUM], mean=0,stddev=0.05), name='g_var')
c_var = tf.Variable(initial_value=tf.random_normal([USER_NUM, CAST_NUM], mean=0,stddev=0.05), name='c_var')

In [12]:
d_result = d_result + d_var
t_result = t_result + t_var
g_result = g_result + g_var
c_result = c_result + c_var

In [13]:
movie_feature_dict = pickle.load(open('../data/'+folder+'/movie_feature_dict.pkl', 'rb'))

In [14]:
#build is_in_matrix, if feature is in the movie, the value in matrix will be 1, otherwise will be 0
d_isin = np.zeros([MOVIE_NUM+1, DIRECTOR_NUM])
t_isin = np.zeros([MOVIE_NUM+1, TOPIC_NUM])
c_isin = np.zeros([MOVIE_NUM+1, CAST_NUM])
g_isin = np.zeros([MOVIE_NUM+1, GENRE_NUM])

for key in movie_feature_dict:
    for f_id in movie_feature_dict[key]['d']:
        d_isin[key,f_id] = 1
    
    for f_id in movie_feature_dict[key]['t']:
        t_isin[key,f_id] = 1
    
    for f_id in movie_feature_dict[key]['c']:
        c_isin[key,f_id] = 1
    
    for f_id in movie_feature_dict[key]['g']:
        g_isin[key,f_id] = 1
    
d_isin = tf.constant(d_isin, tf.float32)
t_isin = tf.constant(t_isin, tf.float32)
c_isin = tf.constant(c_isin, tf.float32)
g_isin = tf.constant(g_isin, tf.float32)

In [15]:
#md_var = tf.Variable(initial_value=tf.truncated_normal([MOVIE_NUM+1, DIRECTOR_NUM]), name='md_var')
#mt_var = tf.Variable(initial_value=tf.truncated_normal([MOVIE_NUM+1, TOPIC_NUM]), name='mt_var')
#mg_var = tf.Variable(initial_value=tf.truncated_normal([MOVIE_NUM+1, GENRE_NUM]), name='mg_var')
#mc_var = tf.Variable(initial_value=tf.truncated_normal([MOVIE_NUM+1, CAST_NUM]), name='mc_var')

In [16]:
md_var = tf.Variable(initial_value=tf.random_normal([MOVIE_NUM+1, DIRECTOR_NUM], mean=1,stddev=0.05), name='md_var')
mt_var = tf.Variable(initial_value=tf.random_normal([MOVIE_NUM+1, TOPIC_NUM], mean=1,stddev=0.05), name='mt_var')
mg_var = tf.Variable(initial_value=tf.random_normal([MOVIE_NUM+1, GENRE_NUM], mean=1,stddev=0.05), name='mg_var')
mc_var = tf.Variable(initial_value=tf.random_normal([MOVIE_NUM+1, CAST_NUM], mean=1,stddev=0.05), name='mc_var')

In [17]:
d_isin = d_isin * md_var
t_isin = t_isin * mt_var
g_isin = g_isin * mg_var
c_isin = c_isin * mc_var

In [18]:
# count how many feature in each movie
movie_feature_num = np.zeros([MOVIE_NUM+1,1])
for key in movie_feature_dict:
    total_num = 0
    for f_key in movie_feature_dict[key]:
        total_num += len(movie_feature_dict[key][f_key])
    movie_feature_num[key] = 1/total_num
feature_num_tensor = tf.tile(movie_feature_num, [1, USER_NUM])
feature_num_tensor = tf.cast(feature_num_tensor, tf.float32)

In [19]:
feature_num_tensor

<tf.Tensor 'Cast:0' shape=(207, 3550) dtype=float32>

In [20]:
d_nm = tf.matmul(d_result, tf.transpose(d_isin))
t_nm = tf.matmul(t_result, tf.transpose(t_isin))
c_nm = tf.matmul(c_result, tf.transpose(c_isin))
g_nm = tf.matmul(g_result, tf.transpose(g_isin))

In [21]:
d_result

<tf.Tensor 'add:0' shape=(3550, 194) dtype=float32>

In [22]:
Total_Rate = (d_nm + t_nm + c_nm + g_nm)*tf.transpose(feature_num_tensor)

In [23]:
clip_U = U.assign(tf.maximum(tf.zeros_like(U), U))
clip_D = D.assign(tf.maximum(tf.zeros_like(D), D))
clip_T = T.assign(tf.maximum(tf.zeros_like(T), T))
clip_C = C.assign(tf.maximum(tf.zeros_like(C), C))
clip_G = G.assign(tf.maximum(tf.zeros_like(G), G))
clip = tf.group(clip_U, clip_D,clip_T,clip_C,clip_G)


In [24]:
total_result_flatten = tf.reshape(Total_Rate, [-1])
TR_R = tf.gather(total_result_flatten, total_train_user * tf.shape(Total_Rate)[1] + total_train_movie, name='total_user_rate')
TR_R_test = tf.gather(total_result_flatten, total_test_user * tf.shape(Total_Rate)[1] + total_test_movie, name='total_user_rate_test')
TR_R_valid = tf.gather(total_result_flatten, total_valid_user * tf.shape(Total_Rate)[1] + total_valid_movie, name='total_user_rate_valid')


In [25]:

d_result_flatten = tf.reshape(d_result, [-1])
t_result_flatten = tf.reshape(t_result, [-1])
g_result_flatten = tf.reshape(g_result, [-1])
c_result_flatten = tf.reshape(c_result, [-1])

D_R = tf.gather(d_result_flatten, d_user * tf.shape(d_result)[1] + d_movie, name='director_user_rate')
T_R = tf.gather(t_result_flatten, t_user * tf.shape(t_result)[1] + t_movie, name='topic_user_rate')
G_R = tf.gather(g_result_flatten, g_user * tf.shape(g_result)[1] + g_movie, name='genre_user_rate')
C_R = tf.gather(c_result_flatten, c_user * tf.shape(c_result)[1] + c_movie, name='cast_user_rate')


In [26]:
total_diff_op = tf.subtract(TR_R, total_train_rating, name='total_trainig_diff')
#total_diff_op_squared = tf.abs(total_diff_op, name="total_squared_difference")
#total_cost = tf.reduce_sum(total_diff_op_squared, name="total_sum_squared_error")/10000 

total_cost = tf.nn.l2_loss(total_diff_op)


d_diff_op = tf.subtract(D_R, d_rating, name='d_trainig_diff')
t_diff_op = tf.subtract(T_R, t_rating, name='t_trainig_diff')
g_diff_op = tf.subtract(G_R, g_rating, name='g_trainig_diff')
c_diff_op = tf.subtract(C_R, c_rating, name='c_trainig_diff')

d_diff_op_squared = tf.abs(d_diff_op, name="d_squared_difference")
t_diff_op_squared = tf.abs(t_diff_op, name="t_squared_difference")
g_diff_op_squared = tf.abs(g_diff_op, name="g_squared_difference")
c_diff_op_squared = tf.abs(c_diff_op, name="c_squared_difference")

d_base_cost = tf.reduce_sum(d_diff_op_squared, name="d_sum_squared_error")/ tf.cast(tf.shape(d_diff_op_squared)[0], tf.float32) * 10000 
t_base_cost = tf.reduce_sum(t_diff_op_squared, name="t_sum_squared_error")/ tf.cast(tf.shape(t_diff_op_squared)[0], tf.float32) * 10000
g_base_cost = tf.reduce_sum(g_diff_op_squared, name="g_sum_squared_error")/ tf.cast(tf.shape(g_diff_op_squared)[0], tf.float32) * 10000
c_base_cost = tf.reduce_sum(c_diff_op_squared, name="c_sum_squared_error")/ tf.cast(tf.shape(c_diff_op_squared)[0], tf.float32) * 10000

base_cost = d_base_cost + t_base_cost + g_base_cost + c_base_cost + total_cost/10
#base_cost = total_cost

In [27]:
lda = tf.constant(.001, name='lambda')

"""
u_norm = tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm') / tf.cast(tf.shape(U)[0], tf.float32)
d_norm = tf.reduce_sum(tf.abs(D, name='director_abs'), name='director_norm')/ tf.cast(tf.shape(D)[1], tf.float32)
t_norm = tf.reduce_sum(tf.abs(T, name='topic_abs'), name='topic_norm')/ tf.cast(tf.shape(T)[1], tf.float32)
g_norm = tf.reduce_sum(tf.abs(G, name='genre_abs'), name='genre_norm')/ tf.cast(tf.shape(G)[1], tf.float32)
c_norm = tf.reduce_sum(tf.abs(C, name='cast_abs'), name='cast_norm')/ tf.cast(tf.shape(C)[1], tf.float32)
"""
u_norm = tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm') 
d_norm = tf.reduce_sum(tf.abs(D, name='director_abs'), name='director_norm')
t_norm = tf.reduce_sum(tf.abs(T, name='topic_abs'), name='topic_norm')
g_norm = tf.reduce_sum(tf.abs(G, name='genre_abs'), name='genre_norm')
c_norm = tf.reduce_sum(tf.abs(C, name='cast_abs'), name='cast_norm')

norm_sums = u_norm + d_norm + t_norm + g_norm + c_norm
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [28]:
#cost = tf.add(base_cost, regularizer)
cost = base_cost + regularizer

In [29]:
lr = tf.constant(.0001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)
#learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

In [30]:
rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(TR_R, total_train_rating))))
rmse_test = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(TR_R_test, total_test_rating))))
rmse_valid = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(TR_R_valid, total_valid_rating))))

In [31]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(100001):
    _, c, r, t, _rmse, _rmse_v, _rmse_t = sess.run([training_step, base_cost, regularizer, total_cost,rmse, rmse_valid, rmse_test])
    sess.run(clip)
    print(i, c,r,t/10, _rmse, _rmse_v, _rmse_t)
    #print(sess.run(md_var)[0][2])

0 250788.0 15.4658 106714.3625 3.66469 3.66284 3.64961
1 187810.0 7.77584 72815.5625 3.02718 3.03681 3.0077
2 183533.0 7.80483 69992.8875 2.96792 2.97744 2.94749
3 179423.0 7.83454 67298.1625 2.91023 2.91964 2.88891
4 175480.0 7.8649 64724.9 2.85405 2.86333 2.83192
5 171706.0 7.89582 62270.2125 2.79941 2.80856 2.77654
6 168109.0 7.92723 59933.1875 2.74637 2.75539 2.72285
7 164694.0 7.95905 57713.3125 2.69503 2.7039 2.67093
8 161464.0 7.99116 55611.21875 2.64549 2.65422 2.62091
9 158407.0 8.02354 53619.29375 2.59768 2.60626 2.5727
10 155532.0 8.05605 51739.2625 2.55173 2.56016 2.52644
11 152833.0 8.08866 49968.3875 2.50768 2.51596 2.48216
12 150300.0 8.12129 48302.2375 2.46552 2.47365 2.43986
13 147920.0 8.15387 46734.278125 2.42518 2.43315 2.39946
14 145681.0 8.18639 45258.63125 2.38658 2.39441 2.3609
15 143575.0 8.21878 43870.921875 2.3497 2.35741 2.32414
16 141589.0 8.25101 42565.33125 2.31448 2.32205 2.28909
17 139713.0 8.28304 41337.0 2.28084 2.2883 2.2557
18 137939.0 8.31484 40180

150 56493.4 11.0174 8151.1625 1.01283 1.05233 1.05057
151 56212.5 11.0313 8093.4 1.00923 1.04888 1.04721
152 55934.5 11.0452 8036.446875 1.00567 1.04548 1.04387
153 55659.2 11.0589 7980.3796875 1.00216 1.04211 1.04058
154 55386.8 11.0726 7925.1359375 0.998684 1.03878 1.03732
155 55117.0 11.0862 7870.72421875 0.995249 1.03549 1.0341
156 54850.0 11.0998 7817.0921875 0.991852 1.03223 1.03092
157 54585.6 11.1132 7764.21484375 0.988494 1.02901 1.02777
158 54324.0 11.1266 7712.159375 0.985174 1.02583 1.02465
159 54065.2 11.1399 7660.8390625 0.98189 1.02268 1.02157
160 53808.9 11.1531 7610.28671875 0.978647 1.01957 1.01853
161 53555.2 11.1663 7560.4734375 0.975436 1.01649 1.01552
162 53304.0 11.1794 7511.37421875 0.972265 1.01345 1.01254
163 53055.3 11.1924 7462.9734375 0.969127 1.01044 1.00959
164 52808.9 11.2053 7415.3015625 0.966028 1.00746 1.00668
165 52565.0 11.2182 7368.32734375 0.962962 1.00452 1.0038
166 52323.6 11.231 7322.0515625 0.959934 1.00161 1.00095
167 52084.6 11.2437 7276.479

290 34980.0 12.3787 4682.3359375 0.767637 0.813417 0.818325
291 34905.1 12.3853 4674.4484375 0.76699 0.812765 0.817712
292 34830.9 12.3918 4666.50546875 0.766337 0.812111 0.817093
293 34757.3 12.3983 4658.79257812 0.765705 0.811473 0.816492
294 34684.3 12.4047 4651.06953125 0.76507 0.810836 0.81589
295 34611.9 12.4112 4643.5046875 0.764448 0.81021 0.815299
296 34540.1 12.4176 4635.99921875 0.763829 0.809587 0.81471
297 34468.9 12.424 4628.61796875 0.763222 0.808979 0.814132
298 34398.3 12.4303 4621.29140625 0.762616 0.808371 0.813556
299 34328.3 12.4366 4614.07265625 0.762022 0.807771 0.812988
300 34258.9 12.4429 4606.9625 0.761434 0.807179 0.812429
301 34190.1 12.4492 4599.99960937 0.760859 0.806598 0.811881
302 34121.9 12.4554 4593.03476562 0.760282 0.806019 0.811333
303 34054.3 12.4616 4586.19921875 0.759717 0.80545 0.810792
304 33987.2 12.4677 4579.42851562 0.759155 0.804886 0.810259
305 33920.8 12.4738 4572.75546875 0.758601 0.804332 0.809731
306 33854.8 12.4799 4566.1640625 0.758

428 28689.3 13.0511 4154.5078125 0.723078 0.768524 0.7763
429 28663.0 13.0546 4152.74453125 0.722923 0.768372 0.776157
430 28636.9 13.0582 4151.45195312 0.72281 0.768256 0.776059
431 28611.0 13.0617 4149.7375 0.722662 0.768109 0.77592
432 28585.2 13.0653 4148.47890625 0.722551 0.767998 0.775826
433 28559.6 13.0687 4146.76171875 0.722403 0.76785 0.775683
434 28534.1 13.0723 4145.53984375 0.722297 0.76774 0.775592
435 28508.9 13.0757 4143.86328125 0.722151 0.767598 0.775457
436 28483.7 13.0792 4142.68046875 0.722048 0.76749 0.775369
437 28458.7 13.0827 4141.01015625 0.721901 0.767351 0.775234
438 28433.9 13.0861 4139.84453125 0.7218 0.767246 0.775147
439 28409.2 13.0895 4138.275 0.721663 0.767112 0.775019
440 28384.7 13.093 4137.07851563 0.721559 0.767007 0.774931
441 28360.3 13.0964 4135.53125 0.721425 0.766873 0.774804
442 28336.1 13.0998 4134.38867188 0.721324 0.766776 0.77472
443 28312.1 13.1031 4132.85546875 0.721191 0.766644 0.774596
444 28288.2 13.1065 4131.78515625 0.721097 0.766

566 26187.4 13.4393 4035.215625 0.712619 0.758676 0.76756
567 26175.0 13.4414 4034.34453125 0.712544 0.758613 0.767488
568 26162.8 13.4438 4034.296875 0.712538 0.758608 0.767501
569 26150.6 13.4459 4033.45390625 0.712465 0.758548 0.767432
570 26138.4 13.4482 4033.35195312 0.712455 0.758539 0.767441
571 26126.4 13.4503 4032.54296875 0.712383 0.758482 0.767373
572 26114.4 13.4526 4032.46640625 0.712376 0.758475 0.767387
573 26102.4 13.4546 4031.61914062 0.712302 0.758415 0.767315
574 26090.5 13.4569 4031.565625 0.712297 0.758411 0.767332
575 26078.6 13.459 4030.7328125 0.712224 0.75835 0.76726
576 26066.8 13.4612 4030.7015625 0.712221 0.758348 0.767279
577 26055.1 13.4633 4029.86875 0.712147 0.758287 0.767205
578 26043.4 13.4655 4029.828125 0.712144 0.758285 0.767224
579 26031.8 13.4676 4029.00703125 0.712072 0.758224 0.767153
580 26020.2 13.4698 4028.96015625 0.712067 0.75822 0.767169
581 26008.7 13.4718 4028.15078125 0.711996 0.758161 0.767103
582 25997.3 13.474 4028.1046875 0.711992 0

704 24922.1 13.6909 3989.56640625 0.708578 0.755757 0.765095
705 24915.3 13.6923 3988.93945312 0.708522 0.755716 0.765034
706 24908.5 13.6939 3989.09375 0.708536 0.755734 0.765076
707 24901.8 13.6953 3988.46875 0.708481 0.755692 0.765016
708 24895.0 13.6969 3988.63085938 0.708495 0.755708 0.765057
709 24888.3 13.6982 3988.00898438 0.70844 0.755669 0.764996
710 24881.7 13.6998 3988.159375 0.708453 0.755681 0.765038
711 24875.0 13.7012 3987.51640625 0.708396 0.755641 0.764972
712 24868.5 13.7028 3987.72617188 0.708415 0.755658 0.765023
713 24861.9 13.7041 3987.05429688 0.708354 0.755614 0.76495
714 24855.3 13.7057 3987.24140625 0.708371 0.755632 0.765001
715 24848.7 13.707 3986.63242188 0.708317 0.755592 0.764935
716 24842.2 13.7086 3986.74453125 0.708327 0.755607 0.764975
717 24835.7 13.7099 3986.2015625 0.708278 0.75557 0.76492
718 24829.1 13.7115 3986.2828125 0.708287 0.755585 0.764955
719 24822.6 13.7128 3985.728125 0.708236 0.755548 0.764901
720 24816.2 13.7144 3985.846875 0.708247 

842 24171.7 13.8658 3962.940625 0.70621 0.754453 0.764108
843 24167.3 13.8666 3962.28046875 0.70615 0.754414 0.764025
844 24162.9 13.8679 3962.52578125 0.706172 0.754436 0.764084
845 24158.6 13.8688 3962.01132813 0.706127 0.754404 0.764023
846 24154.3 13.87 3962.13867188 0.706138 0.75442 0.764063
847 24150.0 13.8709 3961.75117188 0.706103 0.754398 0.764018
848 24145.6 13.8721 3961.82851562 0.706112 0.754406 0.764053
849 24141.3 13.873 3961.41914063 0.706074 0.754389 0.764006
850 24137.0 13.8743 3961.56601563 0.706087 0.754398 0.764048
851 24132.8 13.8751 3961.07265625 0.706043 0.754377 0.763995
852 24128.7 13.8764 3961.31328125 0.706065 0.75439 0.764046
853 24124.5 13.8772 3960.70273438 0.70601 0.754361 0.763979
854 24120.3 13.8785 3961.07578125 0.706043 0.754386 0.764045
855 24116.0 13.8792 3960.36054688 0.70598 0.754347 0.763965
856 24112.0 13.8806 3960.77929688 0.706018 0.754376 0.764037
857 24107.6 13.8813 3960.07070312 0.705954 0.754333 0.763957
858 24103.5 13.8826 3960.42070312 0

980 23666.4 13.9949 3943.68867188 0.704492 0.75375 0.763615
981 23663.2 13.9953 3942.9734375 0.704428 0.753705 0.76351
982 23660.3 13.9965 3943.42226562 0.704468 0.753739 0.763609
983 23657.2 13.9969 3942.7328125 0.704408 0.753701 0.763506
984 23654.1 13.9982 3943.16132813 0.704445 0.75373 0.763606
985 23651.0 13.9985 3942.50039062 0.704386 0.753697 0.763507
986 23648.0 13.9998 3942.884375 0.70442 0.753723 0.763599
987 23644.9 14.0001 3942.25625 0.704365 0.753692 0.763502
988 23641.9 14.0014 3942.62421875 0.704396 0.753715 0.763593
989 23638.9 14.0017 3941.99609375 0.704341 0.753684 0.763496
990 23635.8 14.003 3942.38242188 0.704375 0.753711 0.763588
991 23632.8 14.0033 3941.74023438 0.704318 0.75368 0.763491
992 23629.9 14.0046 3942.1953125 0.704359 0.75371 0.763594
993 23626.8 14.0049 3941.46289062 0.704293 0.753675 0.763485
994 23623.9 14.0062 3942.00351563 0.704342 0.753707 0.7636
995 23620.8 14.0065 3941.20625 0.70427 0.753669 0.763482
996 23617.8 14.0078 3941.74492187 0.704319 0.

1116 23296.7 14.0938 3928.08203125 0.703096 0.753443 0.763413
1117 23294.3 14.0939 3927.34804687 0.70303 0.753398 0.76328
1118 23291.9 14.0951 3927.81953125 0.703072 0.753439 0.763402
1119 23289.4 14.0952 3927.1640625 0.703013 0.753397 0.763284
1120 23286.9 14.0963 3927.54765625 0.703048 0.753433 0.763393
1121 23284.6 14.0965 3926.946875 0.702995 0.753394 0.763284
1122 23282.1 14.0976 3927.3390625 0.70303 0.753429 0.763391
1123 23280.0 14.0977 3926.72265625 0.702974 0.753392 0.763281
1124 23277.3 14.0989 3927.1359375 0.703011 0.753421 0.763388
1125 23275.1 14.099 3926.50078125 0.702955 0.753391 0.763278
1126 23272.8 14.1002 3926.9765625 0.702998 0.75342 0.763393
1127 23270.6 14.1003 3926.24921875 0.702932 0.753386 0.763272
1128 23268.3 14.1015 3926.81757813 0.702983 0.753421 0.7634
1129 23265.9 14.1015 3926.03203125 0.702913 0.753383 0.763272
1130 23263.6 14.1027 3926.5875 0.702962 0.75342 0.763397
1131 23261.2 14.1028 3925.82421875 0.702894 0.753379 0.763269
1132 23258.9 14.104 3926.3

KeyboardInterrupt: 

In [ ]:
sess.run(d_var)[0]

In [ ]:
id_2_genre = pickle.load(open('../data/'+folder+'/id_2_genre.pkl', 'rb'))
id_2_director = pickle.load(open('../data/'+folder+'/id_2_director.pkl', 'rb'))
id_2_cast = pickle.load(open('../data/'+folder+'/id_2_cast.pkl', 'rb'))
genre_2_id = pickle.load(open('../data/'+folder+'/genre_2_id.pkl', 'rb'))
director_2_id = pickle.load(open('../data/'+folder+'/director_2_id.pkl', 'rb'))
cast_2_id = pickle.load(open('../data/'+folder+'/cast_2_id.pkl', 'rb'))

In [ ]:
d_list = sess.run(d_result)
mean_list = np.mean(d_list, axis=0)

with open('director_dtcg.csv','w') as f:
    f_csv = csv.writer(f)
    headers = ['name', 'score']
    f_csv.writerow(headers)
    for i in id_2_director:
        rows = [id_2_director[i], mean_list[i]]
        f_csv.writerow(rows)

In [ ]:
c_list = sess.run(c_result)
mean_list = np.mean(c_list, axis=0)

with open('cast_dtcg.csv','w') as f:
    f_csv = csv.writer(f)
    headers = ['name', 'score']
    f_csv.writerow(headers)
    for i in id_2_cast:
        rows = [id_2_cast[i], mean_list[i]]
        f_csv.writerow(rows)

In [ ]:
g_list = sess.run(g_result)
mean_list = np.mean(g_list, axis=0)

with open('genre_dtcg.csv','w') as f:
    f_csv = csv.writer(f)
    headers = ['name', 'score']
    f_csv.writerow(headers)
    for i in id_2_genre:
        rows = [id_2_genre[i], mean_list[i]]
        f_csv.writerow(rows)

In [ ]:
"""
Generate Training RMSE
"""
g_user_test = g_user
g_item_test = g_movie
g_rate_test = g_rating
g_result_flatten = g_result_flatten
#genre rmse
g_test = tf.gather(g_result_flatten, g_user_test * tf.shape(g_result)[1] + g_item_test, name='g_test')
g_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(g_rate_test, g_test))))

d_user_test = d_user
d_item_test = d_movie
d_rate_test = d_rating
d_result_flatten = d_result_flatten
#genre rmse
d_test = tf.gather(d_result_flatten, d_user_test * tf.shape(d_result)[1] + d_item_test, name='d_test')
d_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d_rate_test, d_test))))

t_user_test = t_user
t_item_test = t_movie
t_rate_test = t_rating
t_result_flatten = t_result_flatten
#genre rmse
t_test = tf.gather(t_result_flatten, t_user_test * tf.shape(t_result)[1] + t_item_test, name='t_test')
t_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(t_rate_test, t_test))))

c_user_test = c_user
c_item_test = c_movie
c_rate_test = c_rating
c_result_flatten = c_result_flatten
#genre rmse
c_test = tf.gather(c_result_flatten, c_user_test * tf.shape(c_result)[1] + c_item_test, name='c_test')
c_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(c_rate_test, c_test))))


rmse = tf.divide((g_rmse + d_rmse + t_rmse + c_rmse), 4)



In [ ]:
"""
Generate Validation RMSE
"""
g_user_v = g_user_v
g_item_v = g_movie_v
g_rate_v = g_rating_v
g_result_flatten = g_result_flatten
#genre rmse
g_v = tf.gather(g_result_flatten, g_user_v * tf.shape(g_result)[1] + g_item_v, name='g_v')
g_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(g_rate_v, g_v))))

d_user_v = d_user_v
d_item_v = d_movie_v
d_rate_v = d_rating_v
d_result_flatten = d_result_flatten
#genre rmse
d_v = tf.gather(d_result_flatten, d_user_v * tf.shape(d_result)[1] + d_item_v, name='d_v')
d_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d_rate_v, d_v))))

t_user_v = t_user_v
t_item_v = t_movie_v
t_rate_v = t_rating_v
t_result_flatten = t_result_flatten
#genre rmse
t_v = tf.gather(t_result_flatten, t_user_v * tf.shape(t_result)[1] + t_item_v, name='t_v')
t_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(t_rate_v, t_v))))

c_user_v = c_user_v
c_item_v = c_movie_v
c_rate_v = c_rating_v
c_result_flatten = c_result_flatten
#genre rmse
c_v = tf.gather(c_result_flatten, c_user_v * tf.shape(c_result)[1] + c_item_v, name='c_v')
c_rmse_v = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(c_rate_v, c_v))))


rmse_v = tf.divide((g_rmse_v + d_rmse_v + t_rmse_v + c_rmse_v), 4)

In [ ]:
sess.run(C)

In [ ]:
#training_trend_f = open('../data/'+folder+'/trends.csv', 'w')
#csv_trend = csv.writer(training_trend_f)
#headers = ['round','train_total', 'train_g', 'train_d', 'train_t', 'train_c', 'valid_total', 'valid_g', 'valid_d', 'valid_t', 'valid_c']
#csv_trend.writerow(headers)
lowest_num = 0
lowest_rmse = 9999999
not_dec_num = 0

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(100001):
    _, c, r = sess.run([training_step, cost, regularizer])
    if i%20 == 0:
        _rmse_v, _g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v  = sess.run([rmse_v, g_rmse_v, d_rmse_v, t_rmse_v, c_rmse_v])
        sess.run(clip)
        if lowest_rmse > _rmse_v + 0.001:
            lowest_rmse = _rmse_v
            lowest_num = i
            not_dec_num = 0
        else:
            not_dec_num += 1
            if not_dec_num > 10:
                break
            
        print(i, c, "current rmsr: ", _rmse_v,  "lowest rmse: ", lowest_rmse, "lowest num: ", lowest_num)
        print(_g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v)
        _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse = sess.run([rmse, g_rmse, d_rmse, t_rmse, c_rmse])
        #w_row = [i, _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse, _rmse_v, _g_rmse_v, _d_rmse_v, _t_rmse_v, _c_rmse_v]
        #csv_trend.writerow(w_row)
    if i%200 == 0:
        print()
        print("Traing RMSR(total, g, d, t, c):", i , _rmse, _g_rmse, _d_rmse, _t_rmse, _c_rmse)
        print()
#training_trend_f.close()

In [ ]:
def generate_matrix(row_num, col_num, user, movie, rating):
    # row is movie infor num
    # col is user num
    matr = [[0 for i in range(col_num)] for j in range(row_num)]
    for i in range(len(user)):
        matr[movie[i]][user[i]] = float(rating[i])
    
    return matr
        
    

In [ ]:
user_v, director_v, topic_v, genre_v, cast_v = sess.run([U,D,T,G,C])

d_mat = generate_matrix(DIRECTOR_NUM, USER_NUM, d_user, d_movie, d_rating)
t_mat = generate_matrix(TOPIC_NUM, USER_NUM, t_user, t_movie, t_rating)
c_mat = generate_matrix(CAST_NUM, USER_NUM, c_user, c_movie, c_rating)
g_mat = generate_matrix(GENRE_NUM, USER_NUM, g_user, g_movie, g_rating)


In [ ]:
len(d_mat)

In [ ]:
id_2_genre = pickle.load(open('../data/'+folder+'/id_2_genre.pkl', 'rb'))
id_2_director = pickle.load(open('../data/'+folder+'/id_2_director.pkl', 'rb'))
id_2_cast = pickle.load(open('../data/'+folder+'/id_2_cast.pkl', 'rb'))
genre_2_id = pickle.load(open('../data/'+folder+'/genre_2_id.pkl', 'rb'))
director_2_id = pickle.load(open('../data/'+folder+'/director_2_id.pkl', 'rb'))
cast_2_id = pickle.load(open('../data/'+folder+'/cast_2_id.pkl', 'rb'))

In [ ]:
def get_rating_list(info_id_list, info_matrix, user_vector, user_id, matrix):
    result_list = []
    for _id in info_id_list:
        if matrix[_id][user_id] > 0:
            result_list.append(matrix[_id][user_id]) 
        else:
            
            _v = info_matrix[:, int(_id)]
            _rating = np.dot(user_vector, _v)
            result_list.append(_rating)
    return result_list

def get_rating_list_train(id_tup_list, matrix):
    result_list = []
    for tup in id_tup_list:
        user_id = tup[0]
        info_id = tup[1]
        result_list.append(matrix[info_id][user_id])

def gen_regr_data(m_id, user_rating_tups, is_training = True):
    x_train = []
    y_train = []
    user_id_train = []
    for tup in user_rating_tups:
        one_user_x = []
        rating = float(tup[1])
        y_train.append(rating)
        
        user_id = int(tup[0])
        user_id_train.append(user_id)
        feature_dict = movie_feature_dict[m_id]
        u_v = user_v[user_id]
        
        d_x = get_rating_list(feature_dict['d'], director_v, u_v,user_id, d_mat)
        g_x = get_rating_list(feature_dict['g'], genre_v, u_v, user_id, g_mat)
        t_x = get_rating_list(feature_dict['t'], topic_v, u_v, user_id, t_mat)
        c_x = get_rating_list(feature_dict['c'], cast_v, u_v, user_id, c_mat)
        #print(user_id, d_x, g_x, t_x, c_x)
        
        
        one_user_x = d_x + g_x + t_x + c_x
        #one_user_x = d_x + g_x + c_x
        x_train.append(one_user_x)
        
        feature_name = []
        feature_name += ["d_" + id_2_director[_id] for _id in feature_dict['d']]
        feature_name += ["g_" + id_2_genre[_id] for _id in feature_dict['g']]
        feature_name += ["t_" + str(_id) for _id in feature_dict['t']]
        feature_name += ["c_" + id_2_cast[_id] for _id in feature_dict['c']]
        feature_name += ["bias"]
        
        
    return x_train, y_train, feature_name, user_id_train
        
            
              

In [ ]:
user_count_list = pickle.load(open('../data/'+folder+'/user_count_list.pkl', 'rb'))

In [ ]:
name_list = ['d', 't', 'c', 'g']
divid_list = []
for name in name_list:
    summ = 0
    total = 0
    for user in user_count_list[name]:
        for key in user_count_list[name][user]:
            summ += user_count_list[name][user][key]
            total += 1

    divid_list.append(summ/total)

# Store result matrix

In [ ]:
_d_result, _t_result, _g_result, _c_result =  sess.run([d_result, t_result, g_result, c_result])

In [ ]:
pickle.dump(_d_result, open('../data/'+folder+'/ranking_matrix_director.pkl', 'wb'))
pickle.dump(_t_result, open('../data/'+folder+'/ranking_matrix_type.pkl', 'wb'))
pickle.dump(_g_result, open('../data/'+folder+'/ranking_matrix_genre.pkl', 'wb'))
pickle.dump(_c_result, open('../data/'+folder+'/ranking_matrix_cast.pkl', 'wb'))

In [ ]:
from sklearn import linear_model
import numpy as np

def select_feature_index(feature, n):
    index_list = [0 for i in range(len(feature))]
    temp_f = [abs(feature[i]) for i in range(len(feature))]
    for i in (range(n)):
        min_index = temp_f.index(max(temp_f))
        index_list[min_index] = 1
        temp_f[min_index] = 0
    
    return index_list

def get_predict_rating_old(user_rating, features):
    return np.mean([user_rating[i] + features[i] for i in range(len(user_rating))])


def get_predict_rating(user_rating, features):
    weight_n = [float(abs(ur)) for ur in user_rating]
    summ = sum(weight_n)
    weight = [ur/summ for ur in weight_n]
    #weight = [1./len(weight_n) for ur in weight_n]
    
    weight_total = sum([weight[i]* (user_rating[i] + features[i]) for i in range(len(weight))])
    
    return weight_total

def get_predict_rating_select(user_rating, features, n):
    if len(user_rating) < n:
        return get_predict_rating_old(user_rating, features)
    else:
        index_list = select_feature_index(user_rating, n)
        account_list = []
        for i in range(len(user_rating)):
            if index_list[i] == 1:
                account_list.append(user_rating[i] + features[i])
        return np.mean(account_list)

def get_predict_rating_count(user_rating, features, user_id, feature_name):
    weight = []
    weight_sum = 0
    offset = 0.01
    for j in range(len(feature_name)-1):
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
                offset = 1/divid_list[0]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
                offset = 1/divid_list[2]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
                offset = 1/divid_list[3]
            elif dict_name == 't':
                feature_key = int(feature_n)
                offset = 1/divid_list[1]
        except:
            feature_key = ''
        try:
            prop = user_count_list[dict_name][user_id][feature_key]
        except:
            prop = 1
        prop = offset * prop
        weight_sum += prop
        weight.append(prop)      
    
    weight = [w/float(weight_sum) for w in weight]
    result = 0
    for i in range(len(user_rating)):
        #name = feature_name[i]
        #dict_name = name.split('_')[0]
        #ppr = (user_rating[i] + features[i]) * weight[i]
        result += (user_rating[i] + features[i]) * weight[i]
    """
    print(user_id)
    for i in range(len(weight)):
        print(feature_name[i], end='')
        print(" %.2f %.2f %.2f" % (user_rating[i], features[i], weight[i]))
    
    """
    
    return result

# Equal weight

In [ ]:
total_predict = np.array([])
total_rating = np.array([])
total_predict_v = np.array([])
total_rating_v = np.array([])
total_predict_tr = np.array([])
total_rating_tr = np.array([])
#train_rmse_compare_F = []
for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print("not in:", m)
        continue
    # tuples is (userid, movierating)
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    
    if len(y_train) < 5:
        print('less than 5:', m)
        continue
    #x_train = x_train[1:2]
    #y_train = y_train[1:2]
    
    #if len(y_train) > 50:
     #   continue
    
    if m in movie_rating_test:     
        user_rating_tups_test = movie_rating_test[m]
        x_test, y_test, _, user_id_test= gen_regr_data(int(m), user_rating_tups_test)
    else:
        x_test = []
        y_test = []
        user_id_test = []
    
    if m in movie_rating_validation:     
        user_rating_tups_validation = movie_rating_validation[m]
        x_valid, y_valid, _, user_id_valid= gen_regr_data(int(m), user_rating_tups_validation)
    else:
        x_valid = []
        y_valid = []
        user_id_valid = []
        
    print(len(x_train), len(x_test), len(x_valid))
    feature = [0 for i in range(len(feature_name)-1)]
    
    for j in range(len(feature_name)-1):
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]

        
        for i in range(len(x_train)):
            feature[j] += (y_train[i] - x_train[i][j])
        
        feature[j] /= len(x_train)
    
    #weight = [1 - abs(f) for f in feature] if len(feature) > 0 else []
    #select_indexs = select_feature_index(feature, 100)
    #select_indexs = [1 for i in range(len(feature))]
    #for i in range(len(feature)):
        #print(feature_name[i], 1/(1-feature[i]))
        #if select_indexs[i] == 1:
            #print(feature_name[i], feature[i], weight[i])
    #print()
    

    predict_t = []
    predict_v = []
    predict_tr = []
    
    for i in range(len(x_train)):
        cc = get_predict_rating_old(x_train[i], feature)
        #print("%.2f %.2f" %(cc, y_train[i]))
        #print()
        predict_tr.append(cc)
    
    for i in range(len(x_test)):
        cc = get_predict_rating_old(x_test[i], feature)
        #print("%.2f %.2f" %(cc, y_test[i]))
        #print()
        predict_t.append(cc)
    

    for i in range(len(x_valid)):
        cc = get_predict_rating_old(x_valid[i], feature)
        predict_v.append(cc)
        #print("%.2f %.2f" %(cc, y_valid[i]))
        #print()
     
    train_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_tr, y_train))))
    valid_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_v, y_valid))))
    test_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_t, y_test))))
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    #train_rmse_compare_F.append([m, len(y_train), len(y_valid),len(y_test), train_rmse, valid_rmse, test_rmse])
    
    total_predict = np.concatenate((total_predict, predict_t), axis = 0)
    total_rating = np.concatenate((total_rating, y_test), axis = 0)
    total_predict_v = np.concatenate((total_predict_v, predict_v), axis = 0)
    total_rating_v = np.concatenate((total_rating_v, y_valid), axis = 0)
    total_predict_tr = np.concatenate((total_predict_tr, predict_tr), axis = 0)
    total_rating_tr = np.concatenate((total_rating_tr, y_train), axis = 0)
    
    #print(feature)
    #print(feature_name)
        
    

In [ ]:
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print("train rmse:", rmse)

In [ ]:
for i in range(len(total_predict)):
    if total_predict[i] < 0:
        total_predict[i] = 0.
    if total_predict[i] > 5:
        total_predict[i] = 5.

for i in range(len(total_predict_v)):
    if total_predict_v[i] < 0:
        total_predict_v[i] = 0.
    if total_predict_v[i] > 5:
        total_predict_v[i] = 5.
        
for i in range(len(total_predict_tr)):
    if total_predict_tr[i] < 0:
        total_predict_tr[i] = 0.
    if total_predict_tr[i] > 5:
        total_predict_tr[i] = 5.

        
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_tr, total_rating_tr))))  
print("train rmse:", rmse)

mae = mean_absolute_error(total_rating_tr, total_predict_tr)
print("train mae:", mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_v, total_rating_v))))  
print("valid rmse:", rmse)

mae = mean_absolute_error(total_rating_v, total_predict_v)
print("valid mae:", mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print("test rmse:", rmse)

mae = mean_absolute_error(total_rating, total_predict)
print("test mae:", mae)
        


# weight with number of featured movies

In [ ]:
total_predict = np.array([])
total_rating = np.array([])
total_predict_v = np.array([])
total_rating_v = np.array([])
total_predict_tr = np.array([])
total_rating_tr = np.array([])
train_rmse_compare_E = []
movie_feature = {}
feature_name_dict = {}
dropped_movie = 0
for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print('not in', m)
        dropped_movie += 1
        continue
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    
    if len(y_train) < 5:
        print('less than 5:', m)
        dropped_movie += 1
        continue
    #if len(y_train) > 50:
     #   continue
    
    if m in movie_rating_test:     
        user_rating_tups_test = movie_rating_test[m]
        x_test, y_test, _, user_id_test= gen_regr_data(int(m), user_rating_tups_test)
    else:
        x_test = []
        y_test = []
        user_id_test = []
    
    if m in movie_rating_validation:     
        user_rating_tups_validation = movie_rating_validation[m]
        x_valid, y_valid, _, user_id_valid= gen_regr_data(int(m), user_rating_tups_validation)
    else:
        x_valid = []
        y_valid = []
        user_id_valid = []
    
    if len(x_test) == 0:
        continue
        
    
    feature = [0 for i in range(len(feature_name)-1)]
    
    for j in range(len(feature_name)-1):
        prop_sum = 0
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        offset = 0.01
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
                offset = 1/divid_list[0]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
                offset = 1/divid_list[2]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
                offsedivid_listt = 1/divid_list[3]
            elif dict_name == 't':
                feature_key = int(feature_n)
                offset = 1/divid_list[1]
        except:
            feature_key = ''
        
        for i in range(len(x_train)):
            try:
                prop = user_count_list[dict_name][user_id_train[i]][feature_key]
            except:
                prop = 1
            prop = offset * prop
            #prop_sum += prop
            #feature[j] += (y_train[i] - x_train[i][j]) * prop
            prop_sum += 1
            feature[j] += (y_train[i] - x_train[i][j])
        
        feature[j] /= prop_sum
    feature_name_dict[m] = feature_name
    movie_feature[m] = feature
    #weight = [1 - abs(f) for f in feature] if len(feature) > 0 else []
    #select_indexs = select_feature_index(feature, 100)
    #select_indexs = [1 for i in range(len(feature))]
    #for i in range(len(feature)):
        #print(feature_name[i], 1/(1-feature[i]))
        #if select_indexs[i] == 1:
            #print(feature_name[i], feature[i], weight[i])
    #print()
    

    predict_t = []
    predict_v = []
    predict_tr = []
    
    for i in range(len(x_train)):
        cc = get_predict_rating_count(x_train[i], feature, user_id_train[i], feature_name)
        #print("%.2f %.2f" %(cc, y_train[i]))
        #print()
        predict_tr.append(cc)
    
    for i in range(len(x_test)):
        cc = get_predict_rating_count(x_test[i], feature, user_id_test[i], feature_name)
        #print("%.2f %.2f" %(cc, y_test[i]))
        #print()
        predict_t.append(cc)
    

    for i in range(len(x_valid)):
        cc = get_predict_rating_count(x_valid[i], feature, user_id_valid[i], feature_name )
        predict_v.append(cc)
        #print("%.2f %.2f" %(cc, y_valid[i]))
        #print()
        
        
    #train_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_tr, y_train))))
    #valid_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_v, y_valid))))
    #test_rmse = np.sqrt(np.mean(np.square(np.subtract(predict_t, y_test))))
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    
    #print(m, train_rmse, valid_rmse, test_rmse)
    #train_rmse_compare_E.append([m, len(y_train), len(y_valid),len(y_test), train_rmse, valid_rmse, test_rmse])
    
        
    total_predict = np.concatenate((total_predict, predict_t), axis = 0)
    total_rating = np.concatenate((total_rating, y_test), axis = 0)
    total_predict_v = np.concatenate((total_predict_v, predict_v), axis = 0)
    total_rating_v = np.concatenate((total_rating_v, y_valid), axis = 0)
    total_predict_tr = np.concatenate((total_predict_tr, predict_tr), axis = 0)
    total_rating_tr = np.concatenate((total_rating_tr, y_train), axis = 0)


In [ ]:
len(total_predict)

In [ ]:
index = 3
for i in range(len(movie_feature[index])):
    print(feature_name_dict[index][i], movie_feature[index][i])

In [ ]:
for i in range(len(total_predict)):
    if total_predict[i] < 0:
        total_predict[i] = 0.
    if total_predict[i] > 5:
        total_predict[i] = 5.

for i in range(len(total_predict_v)):
    if total_predict_v[i] < 0:
        total_predict_v[i] = 0.
    if total_predict_v[i] > 5:
        total_predict_v[i] = 5.
        
for i in range(len(total_predict_tr)):
    if total_predict_tr[i] < 0:
        total_predict_tr[i] = 0.
    if total_predict_tr[i] > 5:
        total_predict_tr[i] = 5.

        
rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_tr, total_rating_tr))))  
print( rmse)

mae = mean_absolute_error(total_rating_tr, total_predict_tr)
print(mae)

rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict_v, total_rating_v))))  
print(rmse)

mae = mean_absolute_error(total_rating_v, total_predict_v)
print( mae)


rmse = np.sqrt(np.mean(np.square(np.subtract(total_predict, total_rating))))  
print( rmse)

mae = mean_absolute_error(total_rating, total_predict)
print(mae)
        


# Get one user's information

In [ ]:
user_id = 1
feature_dict = ['d','t','c', 'g']
one_user_count_list = {}
for type_name in feature_dict:
    one_user_count_list[type_name] = {}
for i in range(len(feature_dict)):
    dict_name = feature_dict[i]
    divid_value = divid_list[i]
    for feature_key in user_count_list[dict_name][user_id]:
        one_user_count_list[dict_name][feature_key] = user_count_list[dict_name][user_id][feature_key]/divid_value
    

In [ ]:
user_movie_rating = {}
user_movie_info = {}

for m in range(0, MOVIE_NUM+1):
    if m not in movie_rating:
        print('not in', m)
        dropped_movie += 1
        continue
    user_rating_tups = movie_rating[m]
    x_train, y_train, feature_name, user_id_train = gen_regr_data(int(m), user_rating_tups)
    if len(y_train) < 5:
        print('less than 5:', m)
        dropped_movie += 1
        continue
    
    one_uesr_ratings = []
    one_user_info = []
    weight = []
    x_pred, _, feature_name, user_id_pred = gen_regr_data(int(m), [(user_id, 0)])
    if user_id in user_id_train:
        ind = user_id_train.index(user_id)
        real_rating = y_train[ind]
    else:
        real_rating = -1
    for j in range(len(feature_name)-1):
        prop_sum = 0
        name = feature_name[j]
        dict_name = name.split('_')[0]
        feature_n = name.split('_')[1]
        offset = 0.01
        try:
            if dict_name == 'd':
                feature_key = director_2_id[feature_n]
            elif dict_name == 'c':
                feature_key = cast_2_id[feature_n]
            elif dict_name == 'g':
                feature_key = genre_2_id[feature_n]
            elif dict_name == 't':
                feature_key = int(feature_n)
        except:
            feature_key = ''
        try:
            weight.append(one_user_count_list[dict_name][feature_key])
        except:
            # if user have never seen this information
            weight.append(0.1)

    weight_normal = [w/sum(weight) for w in weight]
    print('begin')
    try:
        one_pred_value = sum([weight_normal[i]*(x_pred[0][i] + movie_feature[m][i]) for i in range(len(x_pred[0]))])
    except:
        continue
    feature_name_dict
    """
    print("movie:", m)
    print("pred:", one_pred_value)
    print("real:", real_rating)
    
    for i in range(len(x_pred[0])):
        print(feature_name_dict[m][i], x_pred[0][i], movie_feature[m][i], weight_normal[i])
    """
    user_movie_rating[m] = (one_pred_value, real_rating)
    user_movie_info[m] = {}
    user_movie_info[m]['info_name'] = feature_name_dict[m][0:-1]
    user_movie_info[m]['info_rating'] = x_pred[0]
    user_movie_info[m]['movie_offset'] = movie_feature[m]
    user_movie_info[m]['weight'] = weight_normal
    



        

In [ ]:
for m in user_movie_rating:
    if user_movie_rating[m][1] > 0:
        print(m, user_movie_rating[m][1])
        length = len(user_movie_info[m]['info_rating'])
        for i in range((length)):
            print(user_movie_info[m]['info_name'][i], user_movie_info[m]['info_rating'][i] + user_movie_info[m]['movie_offset'][i], user_movie_info[m]['weight'][i])


        

In [ ]:
best_movie = -1
best_rating = 0
for m in user_movie_rating:
    if user_movie_rating[m][1] < 0 and user_movie_rating[m][0] > best_rating:
        best_rating = user_movie_rating[m][0]
        best_movie = m

best_movie

In [ ]:
best_movie = 1817
print('rating:',user_movie_rating[best_movie])
length = len(user_movie_info[best_movie]['info_rating'])
for i in range((length)):
    part = (user_movie_info[best_movie]['info_rating'][i] + user_movie_info[best_movie]['movie_offset'][i]) * user_movie_info[best_movie]['weight'][i]
    print(user_movie_info[best_movie]['info_name'][i], user_movie_info[best_movie]['info_rating'][i], user_movie_info[best_movie]['movie_offset'][i], user_movie_info[best_movie]['weight'][i], part)
    #print(user_movie_info[best_movie]['info_name'][i])
    
    #print(part)
